In [39]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf

class DenseNN(tf.Module):
    def __init__(self, outputs):
        super().__init__()
        self.outputs = outputs
        self.fl_init = False

    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name="b")

            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b)

            self.fl_init = True

        y = x @ self.w + self.b # 
        return y


model = DenseNN(1)

print( model(tf.constant([[1.0, 2.0]])) )

tf.Tensor([[0.14808246]], shape=(1, 1), dtype=float32)


In [41]:
print(model.trainable_variables)

(<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>, <tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[-0.0234493 ],
       [ 0.08576588]], dtype=float32)>)


In [38]:
#print( model(tf.constant([[1.0, 2.0]])) )

x_train = tf.random.uniform(minval=0, maxval=10, shape=(100, 2))
y_train = [a + b for a, b in x_train] 

loss = lambda x, y: tf.reduce_mean(tf.square(x - y))
opt = tf.optimizers.Adam(learning_rate=0.01)

EPOCHS = 20
for n in range(EPOCHS):
    for x, y in zip(x_train, y_train):
        x = tf.expand_dims(x, axis=0)
        y = tf.constant(y, shape=(1, 1)) 
        with tf.GradientTape() as tape:
            f_loss = loss(y, model(x))

        grads = tape.gradient(f_loss, model.trainable_variables)
        opt.apply_gradients(zip(grads, model.trainable_variables))

    print(f_loss.numpy())

print(model.trainable_variables)

0.04922606
0.0010797665
0.002377578
0.0046391655
0.0043057897
0.0031892825
0.0020550727
0.0011779849
0.0006086856
0.00028587005
0.000122044235
4.7056572e-05
1.618897e-05
4.8952643e-06
1.2922683e-06
2.9653256e-07
6.0071216e-08
1.08057066e-08
1.8417268e-09
2.9467628e-10
3.274181e-11
3.637979e-12
9.094947e-13
9.094947e-13
9.094947e-13
9.094947e-13
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
(<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([2.84924e-08], dtype=float32)>, <tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[1.],
       [1.]], dtype=float32)>)


In [ ]:
#[1 - 1] 0 

In [ ]:
#y = f(x) -> loss()

---

In [43]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np


class DenseNN(tf.Module):
    def __init__(self, outputs, activate="relu"):
        super().__init__()
        self.outputs = outputs
        self.activate = activate
        self.fl_init = False

    def __call__(self, x):
        if not self.fl_init:
            self.w = tf.random.truncated_normal((x.shape[-1], self.outputs), stddev=0.1, name="w")
            self.b = tf.zeros([self.outputs], dtype=tf.float32, name="b")

            self.w = tf.Variable(self.w)
            self.b = tf.Variable(self.b, trainable=False)

            self.fl_init = True

        y = x @ self.w + self.b

        if self.activate == "relu":
            return tf.nn.relu(y)
        elif self.activate == "softmax":
            return tf.nn.softmax(y)

        return y


class SequentialModule(tf.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = DenseNN(128)
        self.layer_2 = DenseNN(10, activate="softmax")

    def __call__(self, x):
        return self.layer_2(self.layer_1(x))


(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255
x_test = x_test / 255

x_train = tf.reshape(tf.cast(x_train, tf.float32), [-1, 28*28])
x_test = tf.reshape(tf.cast(x_test, tf.float32), [-1, 28*28])

y_train = to_categorical(y_train, 10)


model = SequentialModule()
# layer_1 = DenseNN(128)
# layer_2 = DenseNN(10, activate="softmax")
#print(model.submodules)

cross_entropy = lambda y_true, y_pred: tf.reduce_mean(tf.losses.categorical_crossentropy(y_true, y_pred))
opt = tf.optimizers.Adam(learning_rate=0.001)

BATCH_SIZE = 32
EPOCHS = 10
TOTAL = x_train.shape[0]

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)


@tf.function
def train_batch(x_batch, y_batch):
    with tf.GradientTape() as tape:
        f_loss = cross_entropy(y_batch, model(x_batch))

    grads = tape.gradient(f_loss, model.trainable_variables)
    opt.apply_gradients(zip(grads, model.trainable_variables))

    return f_loss


for n in range(EPOCHS):
    loss = 0
    for x_batch, y_batch in train_dataset:
        loss += train_batch(x_batch, y_batch)

    print(loss.numpy())


y = model(x_test)
y2 = tf.argmax(y, axis=1).numpy()
acc = len(y_test[y_test == y2])/y_test.shape[0] * 100
print(acc)

acc = tf.metrics.Accuracy()
acc.update_state(y_test, y2)
print( acc.result().numpy() * 100 )

507.08713
214.69572
144.48116
106.609665
82.63441
62.94324
50.177147
40.32832
31.447765
26.596634
97.61
97.61000275611877


In [42]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# Warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")

Eager conv: 0.005080399999314977
Function conv: 0.007838600000468432
Note how there's not much difference in performance for convolutions


---

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10, mnist

tf.random.set_seed(1)


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train / 255
x_test = x_test / 255

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

inputs = keras.Input(shape=(32, 32, 3), name="img")
x = layers.Conv2D(32, 3, activation="relu")(inputs)
x = layers.Conv2D(64, 3, activation="relu")(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(64, 3, activation="relu")(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs, name="toy_resnet")
#model.summary()

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=64, epochs=15, validation_split=0.2)

print( model.evaluate(x_test, y_test) )

170508288/170498071 [==============================] - 40s 0us/step
Epoch 1/15
625/625 [==============================] - 22s 17ms/step - loss: 1.7680 - accuracy: 0.3206 - val_loss: 1.4518 - val_accuracy: 0.4574
Epoch 2/15
625/625 [==============================] - 10s 16ms/step - loss: 1.3189 - accuracy: 0.5200 - val_loss: 1.1677 - val_accuracy: 0.5753
Epoch 3/15
625/625 [==============================] - 10s 15ms/step - loss: 1.0966 - accuracy: 0.6070 - val_loss: 1.0543 - val_accuracy: 0.6269
Epoch 4/15
625/625 [==============================] - 10s 16ms/step - loss: 0.9537 - accuracy: 0.6599 - val_loss: 0.9067 - val_accuracy: 0.6731
Epoch 5/15
625/625 [==============================] - 10s 16ms/step - loss: 0.8557 - accuracy: 0.6976 - val_loss: 0.8841 - val_accuracy: 0.6799
Epoch 6/15
625/625 [==============================] - 9s 14ms/step - loss: 0.7772 - accuracy: 0.7247 - val_loss: 0.7543 - val_accuracy: 0.7337
Epoch 7/15
625/625 [==============================] - 10s 16ms/step -

In [ ]:
#(conv2D) (input)
# 

In [3]:
model.summary()

Model: "toy_resnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 30, 30, 32)   896         ['img[0][0]']                    
                                                                                                  
 conv2d_1 (Conv2D)              (None, 28, 28, 64)   18496       ['conv2d[0][0]']                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 9, 9, 64)     0           ['conv2d_1[0][0]']               
                                                                                         

---

In [5]:
import os
import tensorflow as tf
x =tf.Variable(-2, dtype=tf.float32)   #y = x**2 -> -2 - > 

In [7]:
with tf.GradientTape() as tape:
    y=x**2
df = tape.gradient(y,x)
print(df)


tf.Tensor(-4.0, shape=(), dtype=float32)


In [ ]:
#dy/dx = 2x = 2*(-2) = -4

In [14]:
w =tf.Variable(tf.random.normal((3,2)))
b =tf.Variable(tf.zeros(2,dtype=tf.float32))
x =tf.Variable([[-2.,1.,3.]])

In [15]:
x

<tf.Variable 'Variable:0' shape=(1, 3) dtype=float32, numpy=array([[-2.,  1.,  3.]], dtype=float32)>

In [16]:
with tf.GradientTape() as tape:
    y=x@w + b
    loss =tf.reduce_mean(y**2)
df = tape.gradient(loss,[w,b])
print(df)

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[  7.8689256,  -2.1992188],
       [ -3.9344628,   1.0996094],
       [-11.803389 ,   3.2988281]], dtype=float32)>, None]


In [24]:
x =tf.Variable(0, dtype=tf.float32) #trinable=False
b =tf.Variable(1.5) # Variable #+1 # Constant
with tf.GradientTape() as tape:
    f = (x+b)**2 + 2*b
df = tape.gradient(f,[x,b])

In [25]:
print(df[0])
print(df[1])

tf.Tensor(3.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)


In [26]:
x =tf.Variable([1.,2.]) #trinable=False

with tf.GradientTape() as tape:
    y = tf.reduce_sum([2.,3.]*x**2)
df = tape.gradient(y,x)

In [27]:
print(df)

tf.Tensor([ 4. 12.], shape=(2,), dtype=float32)
